<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/SeoulGrandPark_Prediction_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 8.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.3 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=de450d173eca239a82a1e609525563a8d94b071fa3181bd4cb666413afb26773
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 28 13:35 kaggle.json


In [3]:
!kaggle competitions download -c 2020-ai-termproject-18011817

  0% 0.00/1.14k [00:00<?, ?B/s]
100% 1.14k/1.14k [00:00<00:00, 1.95MB/s]
  0% 0.00/466 [00:00<?, ?B/s]
100% 466/466 [00:00<00:00, 467kB/s]
  0% 0.00/24.3k [00:00<?, ?B/s]
100% 24.3k/24.3k [00:00<00:00, 20.6MB/s]


In [4]:
!unzip 2020-ai-termproject-18011817.zip

unzip:  cannot find or open 2020-ai-termproject-18011817.zip, 2020-ai-termproject-18011817.zip.zip or 2020-ai-termproject-18011817.zip.ZIP.


In [66]:
import pandas as pd
import numpy as np
import random

import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision.datasets as data
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [67]:
device = torch.device('cuda')

torch.manual_seed(777)
random.seed(777)
torch.cuda.manual_seed_all(777)

learning_rate = 0.01
training_epochs = 6000
batch_size = 60

In [68]:
xy_train = pd.read_csv('train_seoul_grandpark.csv', header = None, skiprows=1, usecols=range(1, 8))

x_data = xy_train.iloc[: , 1:-1]
y_data = xy_train.iloc[: , [-1]]

x_data = np.array(x_data)
y_data = np.array(y_data)

scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

x_train.shape

torch.Size([636, 5])

In [69]:
train_dataset = TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           drop_last = True)

In [70]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [71]:
linear1 = torch.nn.Linear(5, 8,bias=True)
linear2 = torch.nn.Linear(8, 8,bias=True)
linear3 = torch.nn.Linear(8, 8,bias=True)
linear4 = torch.nn.Linear(8, 8,bias=True)
linear5 = torch.nn.Linear(8, 1,bias=True)
#dropout = torch.nn.Dropout(p=drop_prob)
mish = Mish()

In [72]:
torch.nn.init.kaiming_normal_(linear1.weight)
torch.nn.init.kaiming_normal_(linear2.weight)
torch.nn.init.kaiming_normal_(linear3.weight)
torch.nn.init.kaiming_normal_(linear4.weight)
torch.nn.init.kaiming_normal_(linear5.weight)

model = torch.nn.Sequential(linear1,mish,
                            linear2,mish,
                            linear3,mish,
                            linear4,mish,
                            linear5).to(device)

In [73]:
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

losses = []
model_history = []
err_history = []

total_batch = len(data_loader)

for epoch in range(training_epochs + 1):
  avg_cost = 0
  #model.train()
  
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = loss(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
    
  model_history.append(model)
  err_history.append(avg_cost)
  
  if epoch % 100 == 0:  
    print('Epoch:', '%d' % (epoch + 1), 'Cost =', '{:.9f}'.format(avg_cost))
  losses.append(cost.item())
print('Learning finished')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


Epoch: 1 Cost = 107973360.000000000
Epoch: 101 Cost = 43472636.000000000
Epoch: 201 Cost = 40194460.000000000
Epoch: 301 Cost = 38392648.000000000
Epoch: 401 Cost = 39578184.000000000
Epoch: 501 Cost = 39945072.000000000
Epoch: 601 Cost = 36810400.000000000
Epoch: 701 Cost = 39775176.000000000
Epoch: 801 Cost = 38688892.000000000
Epoch: 901 Cost = 40333216.000000000
Epoch: 1001 Cost = 40380856.000000000
Epoch: 1101 Cost = 40210608.000000000
Epoch: 1201 Cost = 39921696.000000000
Epoch: 1301 Cost = 40546184.000000000
Epoch: 1401 Cost = 39746612.000000000
Epoch: 1501 Cost = 38492456.000000000
Epoch: 1601 Cost = 39579224.000000000
Epoch: 1701 Cost = 39822792.000000000
Epoch: 1801 Cost = 39722976.000000000
Epoch: 1901 Cost = 39232804.000000000
Epoch: 2001 Cost = 39697800.000000000
Epoch: 2101 Cost = 39273824.000000000
Epoch: 2201 Cost = 39372388.000000000
Epoch: 2301 Cost = 38928664.000000000
Epoch: 2401 Cost = 38656204.000000000
Epoch: 2501 Cost = 36084328.000000000
Epoch: 2601 Cost = 3810

In [74]:
best_model = model_history[np.argmin(err_history)]

In [75]:
xy_test = pd.read_csv('test_seoul_grandpark.csv', header = None, skiprows=1, usecols = range(1, 7))
x_data = xy_test.iloc[:, 1:]
x_data = np.array(x_data)
x_data = scaler.transform(x_data)
x_test = torch.FloatTensor(x_data).to(device)

with torch.no_grad():
    model.eval()     
    predict = best_model(x_test)


In [76]:
submit = pd.read_csv('submit_sample.csv')
submit['Expected'] = submit['Expected'].astype(float)
for i in range(len(predict)):
  submit['Expected'][i] = predict[i]
submit.to_csv('submit.csv', mode = 'w', index = False, header = True)
submit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Id,Expected
0,20170330,12076.124023
1,20180418,4057.726074
2,20190430,12156.786133
3,20180618,10112.790039
4,20160329,11896.807617
5,20170424,13943.920898
6,20180719,3280.721680
7,20170828,19003.154297
8,20180227,2930.058105
9,20160211,4641.748535


In [77]:
!kaggle competitions submit -c 2020-ai-termproject-18011817 -f submit.csv -m "14010974_이기택"

100% 777/777 [00:06<00:00, 112B/s]
Successfully submitted to SejongAI.텀프로젝트.[서울대공원 입장객 수 예측]